In [ ]:
import sys
sys.path.append("../")

# Overview

This notebook illustrates:
- implementing a blackbox function in Paref
- initializing a sequence of Pareto reflections (corresponding to targeted properties of Pareto reflections)
- initializing a stopping criteria and a MOO algorithm
- applying some MOO algorithm to a sequence and blackbox function

# Setup

Let us imagine the following situation: We have a blackbox function (bbf) that we want to optimize multicriterially or, synonymously, multiobjectively (multi objective optimization, MOO). In other words, we are searching for the bbf Pareto front. The bbf can represent a machine for which we can set certain parameters (vector $x \in D$, where D represents the domain of the problem, which we also call design space) and which then reacts measurably via certain target variables (vector $y \in T$, where T represents the co-domain of the problem, which we also call target space). Or the bbf represents a simulation model, where we can also define specifications x for its design and obtain responses y through simulation.
 
In our example here, we choose as bbf the mathematical test function Zitzler-Deb-Thiele N.1 (ZDT-1, reference: Deb, Kalyan; Thiele, L.; Laumanns, Marco; Zitzler, Eckart (2002). "Scalable multi-objective optimization test problems". Proceedings of the 2002 IEEE Congress on Evolutionary Computation. Vol. 1. pp. 825–830. doi:10.1109/CEC.2002.1007032). In the further course of our example, however, we assume that we do not know this function (which makes it a bbf). We can only "test" the bbf qua function calls in places we choose.  

This approach offers the following advantage: ZDT-1 is not only defined analytically, but we also know (analytically) its Pareto front. Thus, we can immediately compare our results with the expected results and see how well our new approach works.
 
In many cases, the general MOO problem is formulated as "find the Pareto front" and not specified more clearly. Many MOO algorithms are accordingly designed to do just that: They identify Pareto-optimal points (i.e. points on the Paretofront), but they do not guarantee the user any further properties (e.g. to identify co-domain corner points of the Paretofront). The package paref now allows exactly that: besides the general property "identified points are Pareto-optimal", paref users can specify further properties of the Paretofront to be identified and construct MOO algorithms based on those defined properties. Let's make it concrete for our example:
 
From our bbf ZDT-1 we want to identify Pareto-optimal solutions that a) represent the co-domain corners of the Pareto front and b) reflect its path in equidistant steps. The first property gives us an answer to the question "in which target area are there Pareto-optimal trade-offs of our bbf at all?". The second property answers the question "what does the path of the bbf Pareto front look like in a grid that I can specify?". Both are quite typical questions that arise in the context of a bbf MOO. Let's get down to work and see how all these considerations transfer into code....
 


## Step: Definition of blackbox function

In our environment, we first need to define how many dimensions our domain (Design Space) and co-domain (Target Space) have. In our example (ZDT-1) we want to map 10 Design Space dimensions to 2 Target Space dimensions. 
Second, we must define how it can retrieve function values at selected points, i.e. we must declare calls to our bbf. Those four information, i.e.
- assignment of vector of design variables to vector of target values (here given by ZDT-1) implemented in the ``__call__`` method
- dimension of design space (here 10)
- dimension of target space (here 2)
- design space definition (here $[0,1]^{10}$)

must be implemented in Parefs\` ``BlackboxFunction``.

    CAUTION: by default, the evaluations of the blackbox function must be stored in the ``self._evaluations``variable and must be of the form [x,y] where x is a one dimensional numpy array representing the design vector and y a one dimensional numpy array representing the corresponding target vector!

The corresponding code looks like this:

In [ ]:
import numpy as np
from paref.black_box_functions.design_space.bounds import Bounds
from paref.interfaces.moo_algorithms.blackbox_function import BlackboxFunction


class ZDT1(BlackboxFunction):
    def __call__(self, x: np.ndarray) -> np.ndarray:
        f1 = x[0]
        g = 1+9/29*np.sum(x[1:])
        h = 1-np.sqrt(f1/g)
        f2 = g*h
        y = np.array([f1, f2])
        self._evaluations.append([x, y])  # store evaluation
        return y

    @property
    def dimension_design_space(self) -> int:
        return 10

    @property
    def dimension_target_space(self) -> int:
        return 2

    @property
    def design_space(self) -> Bounds:
        return Bounds(upper_bounds=np.ones(self.dimension_design_space),
                      lower_bounds=np.zeros(self.dimension_design_space))

We then initialize an instance of this bbf.

In [ ]:
bbf = ZDT1()

The Pareto front of ZDT-1 looks as follows:

In [ ]:
import plotly.graph_objects as go
pareto_points_of_bbf = [i*np.eye(1,bbf.dimension_design_space,0)[0] for i in np.arange(0,1,0.01)]
pareto_front_of_bbf = np.array([bbf(point) for point in pareto_points_of_bbf])
bbf.clear_evaluations()

data = [
            go.Scatter(x=pareto_front_of_bbf.T[0],
                       y=pareto_front_of_bbf.T[1],
                       name='Real Pareto front',
                       line=dict(width=4)
                       ),
        ]
fig = go.Figure(data=data)
fig.update_layout(
            title="Pareto front of ZDT-1",
            width=600,
            height=600,
            plot_bgcolor='rgba(0,0,0,0)',
            legend=dict(
                x=0.2,
                y=0.9,)
        )
fig.show()

Note that in general we are not able to calculate the Pareto front. In this particular case, we can and use this knowledge in order to validate our results.

## Step: Declaration of generic MOO algorithm (genMOO)

In the third step we have to define a generic MOO algorithm genMOO, i.e. an operator that is able to return a set of Pareto points when we release it on a bbf. We do not need to require further properties of the genMOO algorithm. Such algorithms exist in great variety. In our example we choose the *minimization algorithm* ``DifferentialEvolutionMinimizer`` which is already implemented in Paref. 
This minimization algorithm exploits the fact that the underlying test function (ZDT-1) is cheap to sample.
This looks as follows:

In [ ]:
from paref.moo_algorithms.minimizer.differential_evolution_minimizer import DifferentialEvolutionMinimizer

moo = DifferentialEvolutionMinimizer()

In addition to determining which genMOO algorithm we want to use, we must bear in mind one of their fundamental properties: They only provide approximations for Pareto points or converge to Pareto points when used infinitely long or frequently. Accordingly, we have to define a so-called "stop criterion", i.e. a rule when we consider an approximation to be sufficient for our purposes. In the case of our example, we choose the criterion ``MaxIterationsReached`` which tells the MOO algorithm to stop after a specified number of iterations.
We choose a maximum number of 100 iterations.
The implementation lookes as follows:

In [ ]:
from paref.moo_algorithms.stopping_criteria.max_iterations_reached import MaxIterationsReached

stopping_criteria = MaxIterationsReached(max_iterations=12)

## Step: User definition of properties for MOO search

In this step, the central paref added value-add follows: We define further properties that we demand from a - then user-defined - MOO (then parefMOO called) algorithm when we let it loose on the bbf. In our example case, we require the two properties mentioned above 
- (a) to identify the corners of the bbf Pareto front and 
- (b) to reproduce its path in equidistant steps. 

These two properties are each represented within the paref framework by individual Pareto reflections or sequences. Specifically property a) is represented by the sequence of Pareto reflections ``FindEdgePointsSequence``

In [ ]:
from paref.pareto_reflection_sequences.multi_dimensional.find_edge_points_sequence import FindEdgePointsSequence
sequence_edge_points = FindEdgePointsSequence()

while property b) is represented by the sequence ``FillGapsOfParetoFrontSequence2D``

In [ ]:
from paref.pareto_reflection_sequences.two_dimensional.fill_gaps_of_pareto_front_sequence_2d import \
    FillGapsOfParetoFrontSequence2D
sequence_equidistant_path = FillGapsOfParetoFrontSequence2D()

## Step: parefMOO Sequence application and visualization of Pareto front with user defined properties

In the last step, we first apply the parefMOO sequence defined above to the bbf. Note that the evaluations of the bbf are stored within the ``bbf.evaluations`` resp. ``bbf.y`` and ``bbf.x`` property.
In order to apply the MOO to the sequence and bbf, we simply need to call the ``bbf.apply_to_sequence``method!
We first apply the MOO to the ``FindEdgePointsSequence``:

In [ ]:
moo.apply_to_sequence(sequence_pareto_reflections=sequence_edge_points,
                      blackbox_function=bbf,
                      stopping_criteria=stopping_criteria,)

After determining the edge points, we apply the MOO (with the remaining number of evaluations) to the equidistance sequence:

In [ ]:
moo.apply_to_sequence(sequence_pareto_reflections=sequence_equidistant_path,
                      blackbox_function=bbf,
                      stopping_criteria=stopping_criteria,)

In “real life” you may eventually have to be patient – this step may take a while. Once it is done, we can look at the corresponding result in the co-domain (target space):

In [ ]:
data = [
            go.Scatter(x=pareto_front_of_bbf.T[0],
                       y=pareto_front_of_bbf.T[1],
                       name='Real Pareto front',
                       line=dict(width=4)
                       ),
            go.Scatter(x=bbf.y.T[0], y=bbf.y.T[1],
                       mode='markers',
                       marker=dict(size=10),
                       name='Determined Pareto points'
                       ),
        ]
fig = go.Figure(data=data)
fig.update_layout(
            width=600,
            height=600,
            plot_bgcolor='rgba(0,0,0,0)',
            legend=dict(
                x=0.2,
                y=0.9,)
        )
fig.show()